In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 111 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
pd.set_option("display.max_columns", None)
warnings.simplefilter(action="ignore", category = Warning)

In [ ]:
df = pd.read_csv("diabetes.csv")
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

In [ ]:
xgboost_model = XGBClassifier(random_state=17)
xgboost_model.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 17,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

In [ ]:
def cv_func_result(model, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc"],returns=False):
  cv_results = cross_validate(model, X=X, y=y, cv=cv, scoring=scoring)
  for col in scoring:
    if col == "accuracy":
      print("Accuracy test mean = {0}".format(cv_results["test_accuracy"].mean()))
    elif col == "f1":
      print("F1 test mean = {0}".format(cv_results["test_f1"].mean()))
    elif col == "roc_auc":
      print("ROC_AUC test mean = {0}".format(cv_results["test_roc_auc"].mean()))
  if returns:
    return cv_results

cv_func_result(xgboost_model, X, y, cv=5)

Accuracy test mean = 0.7656820303879128
F1 test mean = 0.6459643958047878
ROC_AUC test mean = 0.8295541579315163


#Hiperparametre optimizasyonu yapmadan önceki hatalarımız :
* Accuracy test mean = 0.7656820303879128
* F1 test mean = 0.6459643958047878
* ROC_AUC test mean = 0.8295541579315163

In [ ]:
xgboost_params = {"learning_rate": [0.1, 0.01, 0.001],
                  "max_depth": [3, 5, 8],
                  "n_estimators": [100, 500, 1000],
                  "colsample_bytree": [ 0.3, 0.7, 1]}

def model_best_params(model, params, cv=5, n_jobs=-1, verbose =True, random_state =17):
  model_best_grid = GridSearchCV(model, params, cv=cv, n_jobs = n_jobs, verbose = verbose).fit(X,y)
  for x in model_best_grid.best_params_.keys():
    print('For {0} best param {1}'.format(x, model_best_grid.best_params_[x]))
  
  print("Set with the best parameters of your model...")
  model_final = model.set_params(**model_best_grid.best_params_, random_state = random_state).fit(X,y)
  return model_final

xgboost_final = model_best_params(xgboost_model, xgboost_params)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
For colsample_bytree best param 0.7
For learning_rate best param 0.01
For max_depth best param 3
For n_estimators best param 1000
Set with the best parameters of your model...


Not : colsample_bytree -> değişkenlerden alınacak olan gözlem sayısı

In [ ]:
cv_func_result(xgboost_final, X, y, cv=5)

Accuracy test mean = 0.7786775316187081
F1 test mean = 0.6642459436770524
ROC_AUC test mean = 0.8326484975541579


#Hiperparametre optimizasyonundan sonra hatalarımız:
* Accuracy test mean = 0.7786775316187081
* F1 test mean = 0.6642459436770524
* ROC_AUC test mean =  0.8326484975541579